# Bitacora para el manejo de secuencias ensambladas y búsqueda con *Blastn*

## Para el siguiente ejercicio es necesario tener el Blast+ instalado en la computadora
https://www.ncbi.nlm.nih.gov/guide/data-software/

## Se utilizarán los contigs formados por el ensamblaje que se localizan en 
`bioinformatica_anotacion2019/data/sec_masiva/velvet`

In [1]:
import os
from pandas import Series, DataFrame
import pandas as pd
from Bio import SeqIO, AlignIO, SeqRecord
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import matplotlib.pyplot as plt 

In [2]:
cd ~/Desktop/bioinformatica_anotacion2019/data/sec_masiva/velvet/output_directory/

/Users/migueldelrio/Desktop/bioinformatica_anotacion2019/data/sec_masiva/velvet/output_directory


In [5]:
os.makedirs('img',exist_ok=True)

In [6]:
ls *.f*

contigs.fa


In [3]:
!grep ">" contigs.fa |wc -l

    2969


# Se analizarán con blastn los contigs obtenidos a la base de datos *nt*

Verifique la localización de la base de datos, en este caso se encuentra en `~/DATA/nt/` o corrija si es necesario

In [ ]:
%%bash
#export BLASTDB=~/DATA/nt/
export BLASTDB=/Volumes/Respaldo_completo/Escritorio/bigdata/nt
date  
time blastn -query contigs.fa \
-db /Volumes/Respaldo_completo/Escritorio/bigdata/nt/nt \
-out contigs.tab \
-evalue 1E-6 \
-max_target_seqs 1 \
-num_threads 1 \
-outfmt "6 std sskingdoms stitle staxids sscinames scomnames sblastnames" 
date

In [7]:
ls

Graph       Log         Roadmaps    contigs.fa  stats.txt
LastGraph   PreGraph    Sequences   img/


In [5]:
!head -2 contigs.tab

head: contigs.tab: No such file or directory


In [ ]:
encabezado =("qseqid", "sseqid", "pident", "length", "mismatch", "gapopen","qstart", 
             "qend", "sstart","send", "evalue", "bitscore", "sskingdoms", "stitle", 
             "staxids", "sscinames", "scomnames", "sblastnames")

In [ ]:
ftab=pd.read_table("contigs.tab", header=None , names= encabezado)
ftab.head()

In [ ]:
len(ftab)

In [ ]:
ftab1= ftab.groupby("sskingdoms")["qseqid"].count()
ftab1 = DataFrame(ftab1)
ftab1.describe()

# Cuántos contigs no son eucariotas?

## Guadardo la base de datos en formato csv

In [ ]:
ftab.to_csv("contigs_blastn.csv", header=True, index= None)

### en caso de haber el hecho el análisis previo y querer recuperar el archivo anterior 

In [ ]:
ftab= pd.read_csv("contigs_blastn.csv")
ftab.head(2)

# Hay algún contig con más de una asignación (duplicados)?

In [ ]:
ftab1= ftab.groupby("qseqid")["qseqid"].count()
ftab1 = DataFrame(ftab1)
ftab1

# Es necesario eliminar duplicados

In [ ]:
ftab1 =ftab.drop_duplicates(subset = 'qseqid', keep='first', inplace = False)
ftab1

# Cuántos grupos hay a parte del que pertenece la especie analizada?

In [ ]:
ftab2= ftab1.groupby("sblastnames")["qseqid"].count()
ftab2 = DataFrame(ftab2)
ftab2

In [ ]:
ftab2= ftab1.groupby("sscinames")["qseqid"].count()
ftab2.sort_values(axis = 0, ascending=False, inplace=True)
ftab2

# Procedimiento para simplificar tabla y graficar las 10 primeras categorías y el resto ponerlas en "otras"

In [ ]:
linea10=ftab2[:10]
linea11=ftab2[10:]
#linea10
otro=sum (linea11)
#otro
otros = pd.DataFrame({0:otro}, index=["Other"])
otros
linea10=linea10.append(otros)
linea10

In [ ]:
linea10.plot(kind='barh', figsize= (8,6))
plt.axis([-1, max(linea10[0])+5, -1, 10.8], label=None)
plt.legend().set_visible(False)
plt.xlabel("Frecuencia")
plt.ylabel("Especies")
plt.title("Especies con resultado de blastn")
plt.show()